## Inter-subject correlation

In [ ]:
# Define function that calculates spatial interSC
def calc_spatial_interSC(group_data):

    nGroup = group_data.shape[2]
    
    group_ByScene = np.zeros((nVoxels, nScenes, nGroup))

    for i in range(0, nScenes):
        scene_TRs = (movie_TRs[i])
        scene_TRs = [x - 1 for x in scene_TRs] 
        group_ByScene[:,i,:] = np.mean(group_data[:, scene_TRs, :], axis = 1)
    
    subs = np.arange(0, nGroup)
    group_heldout = np.zeros((nVoxels, nScenes, nGroup))
    
    for i in subs:
        sel_subs = subs[subs!= i]
        group_heldout[:, :, i] = np.mean(group_ByScene[:, :, sel_subs], axis = 2)
    
    if shuffle == 1:
        shuff_scene = list(range(nScenes))
        random.shuffle(shuff_scene)
        group_ByScene = group_ByScene[:, shuff_scene, :] 
        
    full_interSC_group = np.zeros((nScenes, nGroup))
    group_median_nonmatch = np.zeros(nGroup)

    diag_mask = np.ones((nScenes, nScenes), dtype = bool)
    np.fill_diagonal(diag_mask, False)
    div_by = len(diag_mask)

    for i in range(nGroup):
        sub_mat = np.corrcoef(group_ByScene[:,:,i], group_heldout[:,:,i], rowvar = False)
        sub_mat = sub_mat[div_by:, :div_by]
        full_interSC_group[:, i] = np.diagonal(sub_mat)
        group_median_nonmatch[i] = np.median(sub_mat[diag_mask])
    
    return full_interSC_group, group_ByScene, group_median_nonmatch

In [ ]:
# Compute ROI interSC and p-value
shuffle = 0
ROI_data = np.load(f"{npy_dir}/{cROI}_6mm_data.npy")
nVoxels = ROI_data.shape[0]
full_matching_isc, ROI_ByScene, isc_nonmatch = calc_spatial_interSC(ROI_data)
isc_matching = np.median(full_matching_isc, axis = 0)

actual_diff = np.median(isc_matching - isc_nonmatch)
null_dist_diff = np.zeros(nPerms)

shuffle = 1
for k in range(nPerms):
    shuff_full_interSC, shuff_ByScene, shuff_nonmatch = calc_spatial_interSC(ROI_data)
    shuff_interSC = np.median(shuff_full_interSC, axis = 0)
    
    null_dist_diff[k] = np.median(shuff_interSC - shuff_nonmatch)
    perms_above_frac = (len(null_dist_diff[null_dist_diff > actual_diff]) + 1) / (nPerms + 1)
    p_val_diff = min(2*perms_above_frac, 2*(1-perms_above_frac))

In [ ]:
# Calculate median interSC and IQR
matching_isc_med = np.median(isc_matching)
nonmatch_isc_med = np.median(isc_nonmatch)

matching_iqr = iqr(isc_matching)
nonmatch_iqr = iqr(isc_nonmatch)

## InterSC vs. Symptom Severity

In [ ]:
# Define function that correlates an interSC variable with each dimension of a symptom variable
def corr_isc_symp(interSC_var, symp_var): 
    
    actual_corrs = [None] * len(symptom_labels)
    corr_p_vals = [None] * len(symptom_labels)
    all_null_dists = [None] * len(symptom_labels)
    
    for i in range(len(symptom_labels)):
        
        which_score = symptom_labels[i]
        null_dist = np.zeros(nPerms)

        # Actual r value
        x = symp_var[which_score]
        y = interSC_var
        nas = np.logical_or(np.isnan(x), np.isnan(y))
        actual_corr = stats.spearmanr(x[~nas], y[~nas])[0]
        actual_corrs[i] = actual_corr

        # Null dist of r-values 
        for j in range(nPerms):
            x = symp_var[which_score]
            y = interSC_var[random.choices(range(nCohort), k = (nCohort))]
            nas = np.logical_or(np.isnan(x), np.isnan(y))
            null_dist[j] = stats.spearmanr(x[~nas], y[~nas])[0]
    
        perms_above_frac = len((null_dist[null_dist > actual_corr]) + 1) / (nPerms + 1)
        corr_p_vals[i] = min(2*perms_above_frac, 2*(1-perms_above_frac))
        all_null_dists[i] = null_dist
        
    return actual_corrs, corr_p_vals, all_null_dists

In [ ]:
# Correlate interSC with symptom severity
ROI_rho_vals, ROI_p_vals, ROI_nulls =  corr_isc_symp(isc_matching, CAPS_data)

## Final values

In [ ]:
# Assemble final values into list
ROI_values = [matching_isc_med, 
            matching_iqr, 
            nonmatch_isc_med, 
            nonmatch_iqr, 
            actual_diff, 
            p_val_diff]

ROI_values.extend(ROI_rho_vals)
ROI_values.extend(ROI_p_vals)

ROI_value_labels = ['matching_isc_med', 
                    'matching_iqr', 
                    'nonmatch_isc_med', 
                    'nonmatch_iqr', 
                    'diff_score', 
                    'p_val_diff',
                    'rexp_r',
                    'avoid_numb_r',
                    'hyp_r',
                    'rexp_p',
                    'avoid_numb_p',
                    'hyp_p']